# Slice Model
My name for a non-dense model that is just a dense model with set of connections that are *sliced* from layer to layer

## Imports

In [3]:
import arithmetic_datasets as ad

from tensorflow.keras.layers import Input, Lambda, Dense, concatenate
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
import random
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

Using TensorFlow backend.


## SliceModelBuilder

In [25]:
class SliceModelBuilder:
    def __init__(self, input_shape):
        self.input_layer = Input(shape=input_shape)
        self.output_layer = self.input_layer
        
    def dense(self, output_dims):
        self.next_layer = Dense(output_dims)(self.input_layer)
        return self
        
    def slice(self, ranges):
        layers = []
        for rng in ranges:
            layers.append( Lambda(lambda x: x[:,:,rng[0] : rng[1]])(self.next_layer) )
        
        if len(layers) > 1:
            self.next_layer = concatenate(layers)
        else:
            self.next_layer = layers[0]
        
        return self
    
    def make(self):
        return Model(self.input_layer, self.output_layer)

In [26]:
model = SliceModelBuilder((1,3))
model = model.dense(64).dense(32).dense(1).make()
model

In [27]:
model = SliceModelBuilder((1,3)).dense(64).slice([(0,60)]).dense(32).slice([(0,10)]).dense(1).make()
model